Penerapan Algoritma Prim dan Sollin pada jaringan pipa perumahan The Argopuro

Data didapatkan dari PERUMDAM Tirta Pandalungan Jember

Titik merepresentasikan perpotongan pipa dan sisi merepresentasikan pipa dari titik awal ke titik tujuan dengan panjang dari sisi tersebut disebut sebaai bobot

In [ ]:
# import pandas as pd
# import folium as fl
# import heapq
# import networkx as nx
# import time
# from IPython.display import display
# from google.colab import drive
# drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Load the data
df = pd.read_excel('/content/drive/MyDrive/SKRIPSI/Data_Pipa.xlsx')
df.head(3)

,Sisi,TA,Longitude_TA,Latitude_TA,TT,Longitude_TT,Latitude_TT,Bobot
0,e1,v1,113.671379,-8.173614,v3,113.671142,-8.175529,222.12
1,e2,v3,113.671142,-8.175529,v4,113.671262,-8.175522,13.70
2,e3,v2,113.671456,-8.173653,v4,113.671382,-8.175516,214.60


In [ ]:
G = nx.Graph()
# Menambahkan sisi ke graf
for index, row in df.iterrows():
    G.add_edge(row['TA'], row['TT'], weight=row['Bobot'])
# koordinat titik
koordinat_titik = {}
for index, row in df.iterrows():
    koordinat_titik[row['TA']] = (row['Latitude_TA'], row['Longitude_TA'])
    koordinat_titik[row['TT']] = (row['Latitude_TT'], row['Longitude_TT'])

# Visualisasi graf awal
def visualize_on_map(edges, koordinat_titik):
    first_node = list(koordinat_titik.keys())[0] #map berpusat pada titik pertama
    map_center = koordinat_titik[first_node]
    my_map = fl.Map(location=map_center, zoom_start=10)
    #Menambahkan titik pada map
    for i, (node, coords) in enumerate(koordinat_titik.items()):
        html_label = f'<div style="font-weight: bold; font-size: 12pt; ' \
                     f'color:blue;">V{i+1}</div>'
        fl.Marker(coords, icon=fl.features.DivIcon(html=html_label)).add_to(my_map)
        fl.CircleMarker(coords, radius=5, color='yellow', fill=True,
                        fill_color='yellow',fill_opacity=1,popup=node).add_to(my_map)
    #Tambahkan sisi ke map
    edge_counter = 1
    for u, v, weight in edges:
        coords_u = koordinat_titik[u]
        coords_v = koordinat_titik[v]
        line = fl.PolyLine([coords_u, coords_v], color="yellow",
                           weight=2.5, opacity=1)
        line.add_to(my_map)
        midpoint = [(coords_u[0] + coords_v[0])/ 2,(coords_u[1] + coords_v[1])/2]
        edge_label = f'<div style="font-weight: bold; font-size: 14pt; ' \
                     f'color:red;">e{edge_counter}</div>'
        fl.map.Marker(
            midpoint, icon=fl.features.DivIcon(html=edge_label)).add_to(my_map)
        edge_counter += 1
    return my_map

# Visualisasi graf awal
print("Graf awal:")
edges = [(u,v,w) for u,v,w in G.edges(data='weight')]
my_map = visualize_on_map(edges, koordinat_titik)
display(my_map)

# Fungsi untuk menghitung bobot total semua sisi dalam graf
def total_weight(graph):
    return sum(nx.get_edge_attributes(graph, 'weight').values())

# Menghitung bobot total
total_bobot = total_weight(G)
print(f"Total bobot sisi awal graf: {total_bobot}")

Graf awal:


Total bobot sisi awal graf: 5456.599999999999


#Algoritma Prim

In [ ]:
# Ubah data menjadi format graf (dictionary)
G = {}
for _, row in df.iterrows():
    u, v, weight = row['TA'], row['TT'], row['Bobot']
    if u not in G:
        G[u] = {}
    if v not in G:
        G[v] = {}
    G[u][v] = weight
    G[v][u] = weight  # Asumsi graf tidak berarah
G_nx = nx.Graph(G)

# Fungsi Algoritma Prim
def prim(G, start_node):
    E = []  # Daftar sisi dalam Minimum Spanning Tree (MST)
    V = set([start_node])  # Kumpulan simpul yang sudah termasuk dalam MST
    edges = [(weight, start_node, v) for v, weight in G[start_node].items()]
    heapq.heapify(edges)
    while edges and len(V) < len(G):
        weight, u, v = heapq.heappop(edges)
        if v not in V:
            V.add(v)
            E.append((u, v, weight))
            for next_v, next_weight in G[v].items():
                if next_v not in V:
                    heapq.heappush(edges, (next_weight, v, next_v))
    return E
# Jalankan Algoritma Prim
start_node = list(G.keys())[0]
start_time = time.time()
mst_prim = prim(G, start_node)
end_time = time.time()

# Visualisasi mst Prim
def visualize_on_map(mst_prim, koordinat_titik):
    first_node = list(koordinat_titik.keys())[0] #map berpusat pada titik pertama
    map_center = koordinat_titik[first_node]
    my_map = fl.Map(location=map_center, zoom_start=10)
    #Menambahkan titik pada map
    for i, (node, coords) in enumerate(koordinat_titik.items()):
        html_label = f'<div style="font-weight: bold; font-size: 12pt; ' \
                     f'color:blue;">V{i+1}</div>'
        fl.Marker(coords, icon=fl.features.DivIcon(html=html_label)).add_to(my_map)
        fl.CircleMarker(coords, radius=5, color='yellow', fill=True,
                        fill_color='yellow',fill_opacity=1,popup=node).add_to(my_map)
    #Tambahkan sisi ke map
    edge_counter = 1
    for u, v, weight in mst_prim:
        coords_u = koordinat_titik[u]
        coords_v = koordinat_titik[v]
        line = fl.PolyLine([coords_u, coords_v], color="yellow",
                           weight=2.5, opacity=1)
        line.add_to(my_map)
        midpoint = [(coords_u[0] + coords_v[0])/ 2,(coords_u[1] + coords_v[1])/2]
        edge_label = f'<div style="font-weight: bold; font-size: 14pt; ' \
                     f'color:red;">e{edge_counter}</div>'
        fl.map.Marker(
            midpoint, icon=fl.features.DivIcon(html=edge_label)).add_to(my_map)
        edge_counter += 1
    return my_map

# Visualisasi MST Prim
print("Minimum Spanning Tree (Algoritma Prim)")
my_map = visualize_on_map(mst_prim, koordinat_titik)
display(my_map)

# Menghitung total bobot MST
def prim_total_weight(mst_prim): return sum(weight for _, _, weight in mst_prim)
prim_weight = prim_total_weight(mst_prim)
print(f"Total bobot MST: {prim_weight}")

# Cetak hasil MST
print("MST Algoritma Prim:")
for edge in mst_prim: print(f"Sisi: {edge[0]} - {edge[1]}, Bobot: {edge[2]}")
# Cetak waktu eksekusi
elapsed_time = end_time - start_time
print(f"\nWaktu eksekusi: {elapsed_time:.4f} detik")


Minimum Spanning Tree (Algoritma Prim)


Total bobot MST: 4132.429999999999
MST Algoritma Prim:
Sisi: v1 - v3, Bobot: 222.12
Sisi: v3 - v5, Bobot: 13.24
Sisi: v3 - v4, Bobot: 13.7
Sisi: v5 - v6, Bobot: 71.21
Sisi: v5 - v7, Bobot: 139.96
Sisi: v7 - v8, Bobot: 62.64
Sisi: v8 - v16, Bobot: 66.5
Sisi: v16 - v17, Bobot: 52.9
Sisi: v17 - v20, Bobot: 39.95
Sisi: v20 - v36, Bobot: 53.19
Sisi: v36 - v37, Bobot: 61.42
Sisi: v37 - v38, Bobot: 29.6
Sisi: v8 - v9, Bobot: 75.87
Sisi: v9 - v10, Bobot: 22.95
Sisi: v38 - v39, Bobot: 94.62
Sisi: v10 - v11, Bobot: 110.53
Sisi: v11 - v12, Bobot: 21.53
Sisi: v12 - v13, Bobot: 67.64
Sisi: v38 - v40, Bobot: 125.36
Sisi: v40 - v41, Bobot: 57.61
Sisi: v41 - v49, Bobot: 46.0
Sisi: v49 - v50, Bobot: 28.68
Sisi: v50 - v51, Bobot: 23.9
Sisi: v51 - v53, Bobot: 27.35
Sisi: v41 - v42, Bobot: 47.08
Sisi: v42 - v43, Bobot: 42.68
Sisi: v43 - v44, Bobot: 39.31
Sisi: v44 - v45, Bobot: 39.73
Sisi: v45 - v46, Bobot: 40.92
Sisi: v46 - v47, Bobot: 45.97
Sisi: v47 - v48, Bobot: 45.94
Sisi: v51 - v52, Bobot: 62.08
Sis

#Algoritma Sollin

In [ ]:
# Ubah data dari Excel menjadi daftar sisi
solin_data = df[['TA', 'TT', 'Bobot']].values.tolist()
# Buat graf dari data sisi
G = nx.Graph()
G.add_weighted_edges_from(solin_data)

# Algoritma Sollin
def sollin(G):
    edges = sorted(G.edges(data=True), key=lambda x: x[2] # Urutkan sisi besar-kecil
     ['weight'], reverse=True)
    removed_edges = []  # Sisi yang dihapus
    remaining_edges = list(G.edges(data=True)) #Sisi sisa
    for edge in edges: # Loop melalui setiap sisi yang sudah diurutkan
        u, v, w = edge
        def is_connected(G): # Memeriksa apakah graf tetap terhubung
            return nx.is_connected(G)
        G.remove_edge(u, v) # Hapus sisi dari graf
        if not is_connected(G): # Cek apa graf terhubung
            G.add_edge(u, v, weight=w['weight']) # Jika takterhubung, tambahkan sisi
        else:
            removed_edges.append(edge)
            remaining_edges.remove(edge)
    return G, removed_edges, remaining_edges
# Jalankan Algoritma Sollin
start_time = time.time()
mst_sollin, sisi_dihapus, sisi_sollin = sollin(G.copy())
end_time = time.time()

# Fungsi untuk memvisualisasikan graf
def visualize_on_map_sollin(mst_sollin, koordinat_titik):
    first_node = list(koordinat_titik.keys())[0] #map berpusat pada titik pertama
    map_center = koordinat_titik[first_node]
    my_map = fl.Map(location=map_center, zoom_start=10)
    #Menambahkan titik pada map
    for i, (node, coords) in enumerate(koordinat_titik.items()):
        html_label = f'<div style="font-weight: bold; font-size: 12pt; ' \
                     f'color:blue;">V{i+1}</div>'
        fl.Marker(coords, icon=fl.features.DivIcon(html=html_label)).add_to(my_map)
        fl.CircleMarker(coords, radius=5, color='yellow', fill=True,
                        fill_color='yellow',fill_opacity=1,popup=node).add_to(my_map)
    #Tambahkan sisi ke map
    edge_counter = 1
    for u, v, weight in sisi_sollin:
        coords_u = koordinat_titik[u]
        coords_v = koordinat_titik[v]
        line = fl.PolyLine([coords_u, coords_v], color="yellow",
                           weight=2.5, opacity=1)
        line.add_to(my_map)
        midpoint = [(coords_u[0] + coords_v[0])/ 2,(coords_u[1] + coords_v[1])/2]
        edge_label = f'<div style="font-weight: bold; font-size: 14pt; ' \
                     f'color:red;">e{edge_counter}</div>'
        fl.map.Marker(
            midpoint, icon=fl.features.DivIcon(html=edge_label)).add_to(my_map)
        edge_counter += 1
    return my_map

# Visualisasi MST Sollin
print("Minimum Spanning Tree (Algoritma Sollin)")
my_map = visualize_on_map_sollin(sisi_sollin, koordinat_titik)
display(my_map)

# Menghitung total bobot graf MST
def sollin_total_weight(G):
    return sum(nx.get_edge_attributes(G, 'weight').values())
bobot_mst = sollin_total_weight(mst_sollin)
print(f"Total bobot akhir (MST): {bobot_mst}")

# Tampilkan sisi yang dihapus
print("\nSisi yang dihapus:")
for edge in sisi_dihapus:
    print(edge)
# Tampilkan sisi yang tersisa
print("\nSisi yang tersisa:")
for edge in sisi_sollin:
    print(edge)
# Hitung waktu eksekusi
waktu_eksekusi = end_time - start_time
print(f"\nWaktu eksekusi: {waktu_eksekusi:.4f} detik")


Minimum Spanning Tree (Algoritma Sollin)


Total bobot akhir (MST): 4132.429999999999

Sisi yang dihapus:
('v7', 'v37', {'weight': 201.82})
('v34', 'v22', {'weight': 160.24})
('v4', 'v52', {'weight': 143.1})
('v8', 'v13', {'weight': 129.72})
('v9', 'v12', {'weight': 112.52})
('v40', 'v52', {'weight': 109.22})
('v41', 'v47', {'weight': 88.26})
('v48', 'v50', {'weight': 87.27})
('v49', 'v48', {'weight': 84.9})
('v31', 'v30', {'weight': 83.36})
('v32', 'v29', {'weight': 78.56})
('v45', 'v42', {'weight': 45.2})

Sisi yang tersisa:
('v1', 'v3', {'weight': 222.12})
('v3', 'v4', {'weight': 13.7})
('v3', 'v5', {'weight': 13.24})
('v4', 'v2', {'weight': 214.6})
('v5', 'v6', {'weight': 71.21})
('v5', 'v7', {'weight': 139.96})
('v7', 'v8', {'weight': 62.64})
('v8', 'v9', {'weight': 75.87})
('v8', 'v16', {'weight': 66.5})
('v9', 'v10', {'weight': 22.95})
('v10', 'v11', {'weight': 110.53})
('v11', 'v12', {'weight': 21.53})
('v12', 'v13', {'weight': 67.64})
('v13', 'v14', {'weight': 144.22})
('v16', 'v15', {'weight': 291.91})
('v16', 'v17', 